# Even If you wish to run the program multiple times, run the code below this block only once.

In [9]:


# Import all the Libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime 
import time
%matplotlib inline
import matplotlib.pyplot as plt 
import seaborn as sns
import matplotlib.dates as mdates
from mlxtend.frequent_patterns import apriori, association_rules
import warnings
warnings.filterwarnings("ignore")



# Importing the Events File and segregating the events into various columns
events=pd.read_csv("/content/drive/MyDrive/Kaggle/events.csv") 
events_copy=events
events_dummies=pd.get_dummies(events_copy.event)    #To segregate the events into various columns
events_copy_withdummies=pd.concat([events_copy,events_dummies],axis=1)
events_copy_withdummies.drop(["event","timestamp"],axis=1,inplace=True)
events=events_copy_withdummies


#Relating Item_ID with Category_ID

df_item_prop_part1=pd.read_csv("/content/drive/MyDrive/Kaggle/item_properties_part1.csv")
df_item_prop_part2=pd.read_csv("/content/drive/MyDrive/Kaggle/item_properties_part2.csv")
df_itemsprop=df_item_prop_part1.append(df_item_prop_part2)  #Merge both property files to a single file
df_itemsprop.reset_index(inplace=True)
it_cat=df_itemsprop[df_itemsprop.property=="categoryid"].drop(["index","timestamp","property"],axis=1)
it_cat_sort=it_cat.sort_values("itemid").reset_index().drop("index",axis=1)
it_and_cat=it_cat_sort.drop_duplicates()
it_and_cat.reset_index(inplace=True)
for i in it_and_cat.index:
  it_and_cat.at[i,"value"]=int(it_and_cat.iloc[i].value)


# Merging the Category with events
it_and_cat_copy=it_and_cat
events_with_cat=events.merge(it_and_cat_copy,how="inner",on="itemid")
events_with_cat=events_with_cat.rename(columns={'value':'categoryid'})





# Perform the Apriori Algorithm with the Events File


events_with_cat_grouping=events_with_cat[events_with_cat["transactionid"].notna()==True].groupby(["visitorid","categoryid"])["transaction"].sum()
events_with_cat_grouping=events_with_cat_grouping.unstack()
events_with_cat_grouping=events_with_cat_grouping.fillna(0)
def hot_encode(x):
	if(x<= 0):
		return 0
	if(x>= 1):
		return 1
events_with_cat_grouping_encoded=events_with_cat_grouping.applymap(hot_encode)
frq_items = apriori(events_with_cat_grouping_encoded, min_support = 0.0020, use_colnames = True)
# Collecting the inferred rules in a dataframe
rules = association_rules(frq_items, metric ="lift", min_threshold = 1)
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])





# To Form a dataframe outof all the antecedents and cosequents

rules.sort_values(["antecedent support"],ascending=False)
rules_1=rules.drop(["antecedent support","consequent support","support","confidence","lift","leverage","conviction"],axis=1)
rules_1.sort_index(inplace=True)

df_x=pd.DataFrame(None)
df_y=pd.DataFrame(None)
keys={}
for i in events_with_cat["categoryid"].unique():
    keys[i]=0

for i in rules_1.index:
    x=rules_1["antecedents"].iloc[i]
    y=rules_1["consequents"].iloc[i]
    for j in x:
        for k in y:
            df_y.at[keys[j],j]=k
            keys[j]=keys[j]+1
            
df_y=df_y.fillna(-1).astype(int)


df_z=it_and_cat
df_z.rename(columns={'value':'categoryid'},inplace=True)

for i in df_z.categoryid.unique():
    if i not in df_y.columns:
        df_y[i]=-1


# You can run this block of code multiple times


In [22]:

def common(list1,list2):
    intersection=(set(list1).intersection(list2))
    x=list(intersection)
    return x

def commoncategory(li): 
    c_category=[]
    c_category=c_category+(df_y[li[0]]).tolist()
    for i in li:
        apr=df_y[i]
        intersection=common(c_category,apr)
        c_category.clear()
        c_category=c_category+intersection
    return c_category


def takecatinput():
    print("Enter the Category_ID")
    str =  (input ("Enter comma separated integers: "))
    li = []
    listx = str.split (",")
    for z in listx:
        li.append(int(z))
    return li
        
        
        
def takeiteminput():
    print("Enter the Item_ID")
    str =  (input ("Enter comma separated integers: "))
    li = []
    listx = str.split (",")
    for z in listx:
        li.append(int(z))
    
    cat_items=[]
    for j in li:
        cat_items=cat_items+list(df_z[df_z.itemid==j].categoryid.unique())    
    cat_items_unique=set(cat_items)
    x=list(cat_items_unique)
    return x
    
        
def takeinput():
    str=input("Enter 0 for ItemID and 1 for CategoryID")
    x=int(str)
    y=int
    if x==0:
        y=takeiteminput()
        return y
    elif x==1:
        y=takecatinput()
        return y
    else:
        print("Wrong Input. Please Try Again","\n")
        takeinput()

        
        
def finalprocessing(commonapriori):
    final_items=[]    
    for i in commonapriori:
        final_items=final_items+df_z[df_z["categoryid"]==i].itemid.tolist()
    return(finalitems)
    

def all_in_each_cat(catlist):
    final_list=[]
    for i in catlist:
        final_list=final_list+df_z[df_z["categoryid"]==i].itemid.unique().tolist()
    return final_list
    
    

def top5(commonapriori,categorylist):
    if len(commonapriori)>5:
        df_temp=pd.DataFrame(None)
        for j in categorylist:
            for i in rules.index:
                if j in rules.iloc[i].antecedents:
                    df_temp=df_temp.append(rules.iloc[i])
        
        df_temp1=pd.DataFrame(None)
        df_temp.reset_index(inplace=True)
        for i in df_temp.index:
            for j in df_temp.iloc[i].consequents:
                if j in commonapriori:
                    df_temp1=df_temp1.append(df_temp.iloc[i])
                    break
        # print(df_temp1)
        df_temp1.drop("index",axis=1,inplace=True)
        df_temp1.reset_index(inplace=True)
        df_deal=df_temp1
        df_deal.drop("index",axis=1,inplace=True)
       

        confidence={}
        count={}

        for i in commonapriori:
            confidence[i]=0
            count[i]=0

        for i in df_deal.index:
            for j in df_deal.iloc[i].consequents:
                if j in commonapriori:
                    confidence[j]=confidence[j]+df_deal.iloc[i].confidence
                    count[j]=count[j]+1


        commonapriori_conf=[]

        for i in commonapriori:
            commonapriori_conf.append(confidence[i]/count[i])



        conf_to_cat={}
        i=0
        while i<len(commonapriori):
            conf_to_cat[commonapriori_conf[i]]=commonapriori[i]
            i=i+1

        commonapriori_conf.sort(reverse=True)
        commonapriori_conf

        cat_final=[]
        j=0
        while j<5:
            cat_final.append(conf_to_cat[commonapriori_conf[j]])
            j=j+1

        return cat_final
  
    
allcorrectindex=False  


while allcorrectindex==False: 
  allcorrectindex=True       
  categorylist=takeinput()  
  catlistemergency=categorylist.copy()
  commonapriori=[]


  for i in categorylist:
    if df_z[df_z["categoryid"]==i].empty:
      print("Some of the categoryid's that you entered don't exist ")
      allcorrectindex=False




if allcorrectindex==True:
  while len(commonapriori)==0:
      commonapriori=commoncategory(categorylist)
      if -1 in commonapriori:
          commonapriori.remove(-1)
      if len(commonapriori)==0:
          import random
          categorylist.remove(random.choice(categorylist))
      if len(categorylist)==0:
          break

          
  if len(commonapriori)>5:         
    commonapriori=top5(commonapriori,categorylist)


      

  if len(categorylist)!=0:  
      print("The Category Recommendations are: ",commonapriori)
      itemsq=input("Do You Want a list of items in it (Y/N)")
      if itemsq=="Y" or itemsq=="y":
          final_item_list=finalprocessing(commonapriori)    
          print(set(final_itemlist))
      
  else:
      print("Sorry,We Could not Find any category recommendation")
      x=(input("To see the items in the same category as the items you have purchased, Enter 1"))
      if(x=='1'):
          final_item_list=all_in_each_cat(catlistemergency)
          print(set(final_item_list))







Enter 0 for ItemID and 1 for CategoryID0
Enter the Item_ID
Enter comma separated integers: 29,141
The Category Recommendations are:  [959, 1483, 1135, 57, 618]
Do You Want a list of items in it (Y/N)N


In [ ]:
1